In [1]:
with open("klej_polemo2.0-in/dev.tsv", "r") as f:
    raw_dev = f.readlines()

In [2]:
def prepare_data(raw_data):
    corpus = []
    labels = []
    for doc in raw_data:
        text, target = doc.strip().split("\t")
        if "plus" in target:
            label = 0
        elif "minus" in target:
            label = 1
        else:
            label = 2
        corpus.append(text)
        labels.append(label)
    return corpus, labels

In [3]:
test_corpus, test_labels = prepare_data(raw_dev[1:])

# Podejście słownikowe

http://zil.ipipan.waw.pl/SlownikWydzwieku

In [4]:
from collections import Counter
import pandas as pd
from sklearn.metrics import classification_report
import spacy

nlp = spacy.load('pl_core_news_md')

In [5]:
senti_df = pd.read_csv("slownikWydzwieku01.csv", sep="\t", header=None)
senti_df.head()

,0,1,2,3,4,5
0,poddany,c,0,-1,-1,-4.6
1,przewrażliwiony,m,1,1,-2,-3.6
2,cudzoziemiec,c,0,0,0,-2.6
3,przekonywać,m,1,1,0,-1.8
4,skrupuł,c,1,1,1,-3.0


In [6]:
senti_dict = senti_df[[0, 4]].set_index(0).to_dict()[4]
senti_dict

{'poddany': -1,
 'przewrażliwiony': -2,
 'cudzoziemiec': 0,
 'przekonywać': 0,
 'skrupuł': 1,
 'klatka': 0,
 'piękność': 0,
 'osiągnąć': 0,
 'umięśniony': -1,
 'telewizor': 1,
 'ciągnąć': 0,
 'przerazić': 0,
 'ciemnobrązowy': 0,
 'możny': 1,
 'ogolić': 0,
 'dochodowy': 0,
 'piosenka': 0,
 'rząd': -1,
 'towarzystwo': 0,
 'opcja': 0,
 'szklarnia': 1,
 'delikatny': -1,
 'oznaczać': 2,
 'nieobliczalny': 1,
 'omylny': -2,
 'przebić': 2,
 'wiedza': 0,
 'niewystarczający': -1,
 'pracownik': -1,
 'zwrot': 0,
 'satysfakcja': 0,
 'zdziwić': 0,
 'płyn': 0,
 'precyzyjnie': 1,
 'ruchliwy': 1,
 'sklepik': 0,
 'stowarzyszyć': 0,
 'miasto': 0,
 'korupcja': 0,
 'odtłuszczać': 0,
 'pierś': -1,
 'nieciekawy': -2,
 'zadanie': 1,
 'prawdopodobny': 0,
 'całkowicie': 0,
 'nisko': 0,
 'upolitycznić': 0,
 'perfidnie': -2,
 'niezapomniany': 1,
 'stromy': 0,
 'polsat': 0,
 'oczywisty': 0,
 'wliczać': 0,
 'dobrany': 0,
 'cudowny': 0,
 'wiosna': 0,
 'refren': 0,
 'wyrzucać': 0,
 'miejscowy': 1,
 'cios': -1,
 'pent

In [7]:
doc_corpus = list(nlp.pipe(test_corpus, disable=["ner"]))

In [8]:
norm_corpus = [[token.lemma_ for token in doc if token.is_alpha] for doc in doc_corpus]

In [9]:
scores = [[senti_dict.get(token, 0) for token in doc] for doc in norm_corpus]

In [10]:
list(zip(norm_corpus[0], scores[0]))[:20]

[('pani', 0),
 ('doktor', 0),
 ('bardzo', 0),
 ('miły', 2),
 ('ale', 0),
 ('szkło', 0),
 ('dobrać', 0),
 ('mieć', -2),
 ('źle', 0),
 ('w', 0),
 ('gabinet', 0),
 ('być', 0),
 ('ciemno', 1),
 ('odpowiedni', -1),
 ('oświetlenie', 0),
 ('gabinet', 0),
 ('a', 0),
 ('soczewka', 0),
 ('który', 0),
 ('używać', 0)]

In [11]:
scores_sum = [sum(doc) for doc in scores]

In [12]:
scores_avg = [sum(doc) / len(doc) for doc in scores]

In [13]:
max(scores_sum), min(scores_sum)

(21, -20)

In [14]:
max(scores_avg), min(scores_avg)

(0.23636363636363636, -0.17391304347826086)

In [15]:
def predictor(val, pos_t, neg_t):
    if val > pos_t:
        return 0
    elif val < neg_t:
        return 1
    else:
        return 2

In [16]:
Counter(test_labels)

Counter({1: 271, 0: 209, 2: 243})

In [17]:
sum_preds = [predictor(doc, 4, -1) for doc in scores_sum]
Counter(sum_preds)

Counter({2: 336, 0: 229, 1: 158})

In [18]:
avg_preds = [predictor(doc, 0.05, -0.01) for doc in scores_avg]
Counter(avg_preds)

Counter({2: 336, 0: 208, 1: 179})

In [20]:
print(classification_report(test_labels, sum_preds))

              precision    recall  f1-score   support

           0       0.35      0.38      0.37       209
           1       0.49      0.28      0.36       271
           2       0.32      0.44      0.37       243

    accuracy                           0.37       723
   macro avg       0.39      0.37      0.37       723
weighted avg       0.39      0.37      0.37       723



In [21]:
print(classification_report(test_labels, avg_preds))

              precision    recall  f1-score   support

           0       0.43      0.43      0.43       209
           1       0.47      0.31      0.37       271
           2       0.35      0.48      0.40       243

    accuracy                           0.40       723
   macro avg       0.42      0.41      0.40       723
weighted avg       0.42      0.40      0.40       723

